In [36]:
import os
import numpy as np

import IPython.display as ipd

from pedalboard import Compressor, Gain, Limiter, NoiseGate, Pedalboard, Reverb, Distortion, Delay, time_stretch, PitchShift

import librosa
import numpy as np
import matplotlib.pyplot as plt

import pyloudnorm as pyln

In [42]:
def load_audio_files(dir):
    audio_files = []
    for file in os.listdir(dir):
        if file.endswith(".mp3"):
            audio_files.append(os.path.join("/home/ivan/Documents/FIng/QueenMary/", file))
    return audio_files

sr = 44100

base_track_length_s = 60

# load base tracks from /home/ivan/Documents/FIng/QueenMary/
base_tracks_paths = load_audio_files("/home/ivan/Documents/FIng/QueenMary/")
base_tracks = []
for track in base_tracks_paths:
    print(f"Loading {track}...")
    y, _ = librosa.load(track, sr=sr)
    # # cut base song to base_track_length_s
    y = y[:base_track_length_s*sr]
    base_tracks.append(y)

Loading /home/ivan/Documents/FIng/QueenMary/hiphop_1.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/hiphop_3.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/hiphop_2.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/hiphop_4.mp3...


In [43]:
# ipd.Audio(base_tracks[1], rate=sr)

In [44]:
sample_100_path = "/home/ivan/Documents/FIng/QueenMary/sample_100/"
n_samples = 10

# csv columns are sample_id,original_track_id,sample_track_id,t_original,t_sample,n_repetitions,sample_type,interpolation,comments
# open csv and get t_sample
csv_path = os.path.join(sample_100_path, "samples.csv")

sample_tracks = []
sample_tracks_paths = []
with open(csv_path, encoding="latin-1") as f:
    lines = f.readlines()
    for idx in range(n_samples):
        # t_sample = int(lines[idx + 1].split(",")[4])  # +1 because of header of csv
        track = os.path.join(sample_100_path, "audio", lines[idx + 1].split(",")[1]+".mp3")
        sample_tracks_paths.append(track)
        print(f"Loading {track}...")
        y, _ = librosa.load(track, sr=sr)
        sample_tracks.append(y)


Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T002.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T003.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T003.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T005.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T007.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T010.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T011.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T014.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T015.mp3...
Loading /home/ivan/Documents/FIng/QueenMary/sample_100/audio/T018.mp3...


In [45]:
cfg = {
    "sample_start": {
        "seconds": None, # 0,
        "beats": 12
    },
    "sample_length": {
        "seconds": None, # 5,
        "beats": 8
    },
    "gain": {
        "gain_db": -3
    },
    "effects": False, # {
    #     "compressor": {
    #         "threshold_db": -12,
    #         "ratio": 2,
    #         "attack_ms": 0.001,
    #         "release_ms": 0.2
    #     },
    #     "reverb": {
    #         "room_size": 0.8,
    #         "damping": 0.1,
    #         "wet_level": 0.5,
    #         "dry_level": 0.5
    #     },
    #     "limiter": {
    #         "threshold_db": -3
    #     },
    #     "noise_gate": {
    #         "threshold_db": -40,
    #         "attack_ms": 0.001,
    #         "release_ms": 0.1
    #     },
    #     "delay": {
    #         "delay_seconds": 0.5,
    #         "feedback": 0.5,
    #         "mix": 0.5
    #     },
    #     "distortion": {
    #         "drive_db": 0
    #     },
    # },
    "beat_sync": {
        "start_beat": 10, # 0 is the first beat
    },
    "tempo_sync": {
        "multiplier": 1 # 1 is the original tempo of the base track
    },
    "pitch_sync": True, # detects key of the base track and adjusts pitch of sample to match
    "pitch_shift": 0, # in semitones
    "looping": {
        "n_loops": 4, # 1 is no loop
        "loop_every_n_beats": 16
    },
    "masking": True,
}
def effect_sample(sample, sr, cfg):
    assert cfg["gain"] is not None, "Gain is required"
    gain = Gain(gain_db=cfg["gain"]["gain_db"])
    pedalboard = Pedalboard([gain])
    if cfg["effects"]:
        if cfg["effects"]["compressor"]:
            compressor = Compressor(threshold_db=cfg["effects"]["compressor"]["threshold_db"], ratio=cfg["effects"]["compressor"]["ratio"], attack_ms=cfg["effects"]["compressor"]["attack_ms"], release_ms=cfg["effects"]["compressor"]["release_ms"])
            pedalboard.append(compressor)
        if cfg["effects"]["reverb"]:
            reverb = Reverb(room_size=cfg["effects"]["reverb"]["room_size"], damping=cfg["effects"]["reverb"]["damping"], wet_level=cfg["effects"]["reverb"]["wet_level"], dry_level=cfg["effects"]["reverb"]["dry_level"])
            pedalboard.append(reverb)
        if cfg["effects"]["limiter"]:
            limiter = Limiter(threshold_db=cfg["effects"]["limiter"]["threshold_db"])
            pedalboard.append(limiter)
        if cfg["effects"]["noise_gate"]:
            noise_gate = NoiseGate(threshold_db=cfg["effects"]["noise_gate"]["threshold_db"], attack_ms=cfg["effects"]["noise_gate"]["attack_ms"], release_ms=cfg["effects"]["noise_gate"]["release_ms"])
            pedalboard.append(noise_gate)
        if cfg["effects"]["delay"]:
            delay = Delay(delay_seconds=cfg["effects"]["delay"]["delay_seconds"], feedback=cfg["effects"]["delay"]["feedback"], mix=cfg["effects"]["delay"]["mix"])
            pedalboard.append(delay)
        if cfg["effects"]["distortion"]:
            distortion = Distortion(drive_db=cfg["effects"]["distortion"]["drive_db"])
            pedalboard.append(distortion)

    return pedalboard.process(sample, sr)


def loudness_normalize(y, sr, target_loudness=-12.0):
    # peak_normalized_y = pyln.normalize.peak(y, -1.0)
    meter = pyln.Meter(sr, block_size=0.200) # create BS.1770 meter, default block size is 400ms
    loudness = meter.integrated_loudness(y)
    y = pyln.normalize.loudness(y, loudness, target_loudness)
    return y

In [46]:
# create song by adding sample to base
def create_song(base, sample_source, cfg=None):
    # print(f"Peak value of base: {np.max(base)}, Peak value of sample: {np.max(sample)}")
    sample_tempo, sample_beats = librosa.beat.beat_track(y=sample_source, sr=sr)
    base_tempo, base_beats = librosa.beat.beat_track(y=base, sr=sr)
    assert cfg is not None, "Configuration is required"

    sample_time_annotation = []
    base_time_annotations = []

    # extract sample from source
    print("Extracting sample from sample source...")
    assert cfg["sample_start"] is not None, "Sample start is required"
    assert cfg["sample_length"] is not None, "Sample length is required"
    if cfg["sample_start"]["seconds"] is not None:
        sample_source = sample_source[cfg["sample_start"]["seconds"]*sr:]
        sample_time_annotation.append({"start_time": cfg["sample_start"]["seconds"], "end_time": None})
    elif cfg["sample_start"]["beats"] is not None:
        sample_source = sample_source[librosa.frames_to_samples(sample_beats[cfg["sample_start"]["beats"]]):]
        sample_beats -= sample_beats[cfg["sample_start"]["beats"]]
        sample_time_annotation.append({"start_time": librosa.frames_to_time(sample_beats[cfg["sample_start"]["beats"]], sr=sr), "end_time": None})
    if cfg["sample_length"]["seconds"] is not None:
        sample = sample_source[:cfg["sample_length"]["seconds"]*sr]
        sample_time_annotation[-1]["end_time"] = cfg["sample_length"]["seconds"]
    elif cfg["sample_length"]["beats"] is not None:
        sample = sample_source[:librosa.frames_to_samples(sample_beats[cfg["sample_start"]["beats"]+cfg["sample_length"]["beats"]])]
        sample_time_annotation[-1]["end_time"] = librosa.frames_to_time(sample_beats[cfg["sample_start"]["beats"]+cfg["sample_length"]["beats"]], sr=sr)
            
    # loudness normalize audio to -12 dB LUFS
    base = loudness_normalize(base, sr, target_loudness=-12.0)
    sample = loudness_normalize(sample, sr, target_loudness=-12.0)
    original_sample = sample

    # half the volume
    gain = Gain(-6)

    board = Pedalboard([gain])

    base = board.process(base, sr)
    sample = board.process(sample, sr)

    # base_start_sample = 0
    pitch_shift_semitones = 0

    if cfg["masking"]:
        # take the signal to silence for n beats
        pass

    if cfg["tempo_sync"]:
        stretch_rate = (base_tempo/sample_tempo)[0] * cfg["tempo_sync"]["multiplier"]
        sample = time_stretch(sample, sr, stretch_rate)[0]

    if cfg["pitch_sync"]:
        sample_chromagram = librosa.feature.chroma_stft(y=sample, sr=sr)
        base_chromagram = librosa.feature.chroma_stft(y=base, sr=sr)
        mean_sample_chroma = np.mean(sample_chromagram, axis=1)
        mean_base_chroma = np.mean(base_chromagram, axis=1)
        sample_estimated_key = np.argmax(mean_sample_chroma)
        base_estimated_key = np.argmax(mean_base_chroma)
        pitch_shift_semitones = base_estimated_key - sample_estimated_key

    if cfg["pitch_shift"]:
        pitch_shift_semitones += cfg["pitch_shift"]

    pitch_shift = Pedalboard([PitchShift(semitones=pitch_shift_semitones)])
    sample = pitch_shift.process(sample, sr)

        
    print("Applying effects to sample...")
    sample = effect_sample(sample, sr, cfg)
    effected_sample = sample

    mix = base
    # mix[:len(sample)] += sample
    assert cfg["looping"] is not None, "Looping configuration is required (even if n_loops=1)"
    start_beat = cfg["beat_sync"]["start_beat"]
    for _ in range(cfg["looping"]["n_loops"]):
        loop_start_sample = librosa.frames_to_samples(base_beats[start_beat])
        mix[loop_start_sample:loop_start_sample+len(sample)] += sample

        start_beat += cfg["looping"]["loop_every_n_beats"]

        base_time_annotations.append({"start_time": librosa.frames_to_time(base_beats[start_beat], sr=sr), "end_time": librosa.frames_to_time(base_beats[start_beat]+librosa.samples_to_frames(len(sample)), sr=sr)})

        if cfg["looping"]["n_loops"] != 1 and cfg["looping"]["loop_every_n_beats"] < cfg["sample_length"]["beats"]:
            print ("Warning: Loops will overlap as sample_length in beats is greater than loop_every_n_beats")

        # # # find closest beat to start_sample
        # start_beat += cfg["looping"]["loop_every_n_beats"]
        # # loop_start_sample = librosa.frames_to_samples(base_beats[end_beat + cfg["looping"]["loop_every_n_beats"]])
        # loop_start_sample = librosa.frames_to_samples(base_beats[start_beat])


    mix[len(sample):] += base[len(sample):]
    # mix[base_start_sample+len(sample):] += base[base_start_sample+len(sample):]

    mix = loudness_normalize(mix, sr, target_loudness=-12.0)

    return mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation

In [ ]:
import soundfile as sf
import json

for i, (base, sample) in enumerate(zip(base_tracks[:4], sample_tracks[:8:2])):
        mix, original_sample, effected_sample, base_time_annotations, sample_time_annotation = create_song(base, sample, cfg)
        print("====================================")
        print("Original sample")
        ipd.display(ipd.Audio(original_sample, rate=sr))
        print("Sample after processing")
        ipd.display(ipd.Audio(effected_sample, rate=sr))
        print("Mix (final song)")
        ipd.display(ipd.Audio(mix, rate=sr))

        # save mixed audio together with config used
        # save audio
        sf.write(f"mix_{i}_{i*2}.wav", mix, sr)

        # add time annotations to config
        cfg["base_time_annotations"] = base_time_annotations
        cfg["sample_time_annotations"] = sample_time_annotation
        # add base file name to config
        cfg["base_file"] = base_tracks_paths[i]
        cfg["sample_file"] = sample_tracks_paths[i*2]
        # save config
        with open(f"config_{i}_{i*2}.json", "w") as f:
            json.dump(cfg, f)